<center><font size="6">Хакатон от Моторики</font>

[Третий этап](https://www.kaggle.com/competitions/motorica-advanced-gesture-classification)

Команда 1

**Задача:**  Построить классификационную пмодель, которая по показаниям от оптомиографических датчиков будет определять жест оператора протеза кисти во время записи данных, что отражено в значении целевой переменной. 

Имеется три оператора, выполнявшие некоторую последовательность жестов. В результате их работы получены показания датчиков в виде непрерывного временного ряда. 

Список файлов, в котором символ * обозначает индекс оператора:
* X_train_*.npy - обучающая выборка, имеющая размерности "наблюдения, датчики, время";
* X_test_dataset_*.pkl - тестоваая выборка имеющая размерности "наблюдения, датчики, время";
* sample_submission.csv - некоторое решение задачи из которого можно взять формат загрузки данных на Kaggle;
* y_train_*.npy - целевая переменная для обучающей выборки, которая содержит следующие классы:

| Номер <br> класса | Описание жеста |
| :---: | :--- |
| "-1"  |  потенциально "битые" данные |
| "0"  |  жест "open" - разгиб всех пальцев |
| "1"  |  жест "пистолет" |
| "2"  |  сгиб большого пальца |
| "3"  |  жест "ok" |
| "4"  |  жест "grab" |

* functions.py - библиотека функций, применяемых в данном ноутбуке.

# Import библиотек и описание функций

In [1]:
# Импортируем библиотеки
import pandas as pd
import numpy as np

# графические библиотеки
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# библиотеки машинного обучения
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# отображать по умолчанию длину дата Датафрейма
pd.set_option("display.max_rows", 9, "display.max_columns", 9)

# библиотека взаимодействия с интерпретатором

import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
import os

In [36]:
# библиотека вызова функций
import functions

*Пример вызова функции из файла*

In [28]:
functions.fib(100)

0 1 1 2 3 5 8 13 21 34 55 89 


# Загрузка данных
---

In [ ]:
#загрузка обучающей выборки и меток классов
X_train = np.load('X_train.npy')
y_train = pd.read_csv('y_train.csv', index_col='Id').reset_index(drop = True)

#загрузка тестовой выборки
X_test = np.load('X_test.npy')

print(f"Размерность файла 'X_train': {X_train.shape}. Размерность: наблюдения, датчики, время")
print(f"Размерность файла 'y_train': {y_train.shape}")